# Real-Time Experimental Results Visualizer

## Setup

In [ ]:
import pymem.utils.pipeline as pipeline
import plotly.graph_objects as go

In [ ]:
fv_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Formula-Value Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Formula Size (MB)")
    )
)
fv_fig.add_scatter(name='Formula-Value (MB)')
fv_fig

In [ ]:
vo_fig = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Value-Only Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Value Size (MB)")
    )
)
vo_fig.add_scatter(name='Value-Only (MB)')
vo_fig

In [ ]:
barplt = go.FigureWidget(
    layout=go.Layout(
        title=go.layout.Title(text="Live Data"),
        xaxis=dict(title="Rows"),
        yaxis=dict(title="Size (MB)")
    )
)
barplt.add_bar(name='Value-Only (MB)')
barplt.add_bar(name='Formula-Value (MB)')
barplt

## Place your own settings here

In [ ]:
# Input and output paths are relative to root directory
INTEGER_ARG = 1
OUTPUT_PATH = os.path.join("experiments", "results", "excel", "TEST")
SOFFICE_DIR = "C:/Program Files/LibreOffice/program/soffice"
CONFIG_ARGS = pipeline.ConfigArgs(inst="CompleteBipartiteSum"
    , path=os.path.join("experiments", "input-data", "rscs-test")
    , rand=42
    , xlsx=True
    , step=10000
    , rows=0
    , cols=1
    , itrs=2
    , pool=1
)

## Run the following cell for real-time results

In [ ]:
pipeline.run_vis(CONFIG_ARGS
  , OUTPUT_PATH
  , INTEGER_ARG
  , SOFFICE_DIR
  , fv_fig
  , vo_fig
  , barplt
  , 'uss'
)

In [ ]:
# Timing notes

# EXCEL:
# large-RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 06:56:18
# RCBS-1col (Python)
#   Up to 1 million: Total time (HH:MM:SS): 07:05:28
# RCBS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 07:47:56
# RLS-1col
#   Up to 1 million: Total time (HH:MM:SS): 00:06:18
# RSCS-1col (VBA)
#   Up to 1 million: Total time (HH:MM:SS): 00:08:18
# RLS-5col
#   Up to 1 million: Total time (HH:MM:SS): 00:21:21